In [61]:
using DynamicPolynomials, SparseArrays
println("***Problem setting***")

n=100
println("Number of variables: n=",n)

@polyvar x[1:n]# variables

function generate_random_poly(v)
    c=2*rand(Float64,length(v)).-1
    return c'*v
end

function generate_sparse_random_poly(v)
    c=sprand(Float64,length(v),0.01)
    I,V=findnz(c)
    V=2*V.-1
    return sparsevec(I,V,length(v))'*v
end
# random quadratic objective function f
v=reverse(monomials(x,0:2))
f=generate_random_poly(v)


# unit sphere constraint
m=ceil(Int64, n/5)
g=Vector{Polynomial{true,Float64}}(undef,m)
 #type of coefficients of each polynomial must be float

randx=rand(n)# create a feasible solution
randx=rand(1)[1]*randx./sum(randx)
g[1]=1.0-sum(x)
for j in 2:m
    g[j]=generate_random_poly(v[2:end])
    g[j]+=0.125-g[j](x => randx) #make constraints feasible
end



m=length(g)
println("Number of inequality constraints: m=",m)

l=ceil(Int64,n/5)

h=Vector{Polynomial{true,Float64}}(undef,l)
for j in 1:l
    h[j]=generate_random_poly(v[2:end])
    h[j]-=h[j](x => randx) #make constraints feasible
end

#h=[h;1.0-sum(x.^2)]

l=length(h)
println("Number of equality constraints: l=",l)

***Problem setting***
Number of variables: n=100
Number of inequality constraints: m=20
Number of equality constraints: l=20


In [64]:
include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=false);
#data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
#include(data*"/densePOPvar$(n)nineq$(m)neq$(l).jl");

k=3
s=1

@time opt_val1=InterRelax.RelaxDense_without_multiplier(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,
    lmon_f,supp_f,coe_f,dg,dh,k,s,solver="Mosek");

**Interrupted relaxation based on Handelman's Positivstellensatz**
Relaxation order: k=3
Sparsity order: s=1
Maximal matrix size:1
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : LO (linear optimization problem)
  Constraints            : 176851          
  Cones                  : 0               
  Scalar variables       : 185942          
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.12            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 3.21    
Problem
  Name  

135000    64.84       NA          0.00e+00    NA                    -9.091970003253e-01   3229.71 
137500    64.86       NA          0.00e+00    NA                    -9.092258934403e-01   3290.91 
140000    64.86       NA          0.00e+00    NA                    -9.092591396810e-01   3370.06 
142500    64.82       NA          0.00e+00    NA                    -9.092951886162e-01   3449.33 
145000    64.86       NA          0.00e+00    NA                    -9.093241205354e-01   3529.68 
147500    64.85       NA          0.00e+00    NA                    -9.093614251127e-01   3601.53 
150000    64.83       NA          0.00e+00    NA                    -9.093913532515e-01   3667.46 
152500    64.87       NA          0.00e+00    NA                    -9.094171693748e-01   3728.87 
155000    64.86       NA          0.00e+00    NA                    -9.094414009789e-01   3797.67 
157500    64.85       NA          0.00e+00    NA                    -9.094667122595e-01   3874.76 
160000    

InterruptException: InterruptException:

In [ ]:
using SparseArrays
using TSSOS

k=3
pop=[[f(x=>x.^2)];[g[j](x=>x.^2) for j in 1:m];[h[j](x=>x.^2) for j in 1:l]]
@time opt,sol,data=TSSOS.cs_tssos_first(pop,x,k,numeq=l,CS=false,TS="block",solver="Mosek");

************************TSSOS************************
TSSOS is launching...
------------------------------------------------------
The clique sizes of varibles:
[100]
[1]
------------------------------------------------------
Starting to compute the block structure...


In [59]:
opt,sol,data = TSSOS.cs_tssos_higher!(data, TS="block");

Starting to compute the block structure...
No higher CS-TSSOS hierarchy!


In [53]:
include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=false);
#data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
#include(data*"/densePOPvar$(n)nineq$(m)neq$(l).jl");

k=0
s=5

@time opt_val1,opt_sol=InterRelax.RelaxDense(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,
    lmon_f,supp_f,coe_f,dg,dh,k,s,solver="Mosek",comp_opt_sol=false);

**Interrupted relaxation based on Putinar-Vasilescu's Positivstellensatz**
Relaxation order: k=0
Sparsity order: s=5


Maximal matrix size:5
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 231             
  Cones                  : 0               
  Scalar variables       : 239             
  Matrix variables       : 17              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : 

In [6]:
include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=false);
#data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
#include(data*"/densePOPvar$(n)nineq$(m)neq$(l).jl");

k=1
s=31

@time opt_val1,opt_sol=InterRelax.RelaxDense(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,
    lmon_f,supp_f,coe_f,dg,dh,k,s,solver="Mosek",comp_opt_sol=true);

**Interrupted relaxation based on Putinar-Vasilescu's Positivstellensatz**
Relaxation order: k=1
Sparsity order: s=31


Maximal matrix size:4
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 20              
  Cones                  : 0               
  Scalar variables       : 15              
  Matrix variables       : 5               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :          

In [8]:
using SparseArrays

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);
#data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
#include(data*"/densePOPvar$(n)nineq$(m)neq$(l).jl");

using TSSOS

k=2
vars,pop=get_POP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)
@time opt,sol,data=TSSOS.cs_tssos_first(pop,vars,k,numeq=l,CS=false,TS=false,solver="Mosek")

**Semidefinite relaxation based on Putinar's Positivstellensatz**
Relaxation order: k=2
Maximal matrix size: 10


************************TSSOS************************
TSSOS is launching...
------------------------------------------------------
The clique sizes of varibles:
[3]
[1]
------------------------------------------------------
Assembling the SDP...
There are 35 affine constraints.
SDP assembling time: 0.000296825 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 35              
  Cones                  : 0               
  Scalar variables       : 11              
  Matrix variables       : 5               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - 

0.07743037809945671

In [ ]:
using SparseArrays

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);

InterRelax.save_info_densePOP(randx,n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh)